In [118]:
import pandas as pd
import numpy as np
from scipy import stats
import pylab as pl
import matplotlib.patches as mpatches
import requests
import random
from bs4 import BeautifulSoup as bs

In [209]:
url = 'http://tappedout.net/mtg-decks/zur-storm-2/'
cmcurl = url + '?cat=cost&sort='
typeurl = url + '?cat=type&sort='
caturl = url +  '?cat=custom&sort='
cmc = requests.get(cmcurl)
types = requests.get(typeurl)
cat = requests.get(caturl)
cmcsoup = bs(cmc.text, 'html.parser')
typesoup = bs(types.text, 'html.parser')
catsoup = bs(cat.text, 'html.parser')
cmccards = cmcsoup.find_all("a", class_="qty board")
typecards = typesoup.find_all('a', class_='qty board')
catcards = catsoup.find_all("a", class_="qty board")

In [211]:
cardattr = {};
for item in cmccards:
    if item['data-name'] not in cardattr.keys():
        cardattr[item['data-name']] = [int(item['data-category'][0]), int(item['data-qty'])]
for item in typecards:
    cardattr[item['data-name']].append(item['data-category'])
for item in catcards:
    if len(item['data-category']) == 1:
        cardattr[item['data-name']].append(int(item['data-category']))
    else:
        cardattr[item['data-name']].append(item['data-category'])

In [212]:
cardattr

{'Ad Nauseam': [5, 1, 'instant', 0],
 'Aetherflux Reservoir': [4, 1, 'artifact', 0],
 'Ancient Tomb': [0, 1, 'land', 0],
 "Angel's Grace": [1, 1, 'instant', 'combo'],
 'Arid Mesa': [0, 1, 'land', 0],
 'Azorius Signet': [2, 1, 'artifact', 6],
 'Bloodstained Mire': [0, 1, 'land', 0],
 'Brainstorm': [1, 1, 'instant', 'draw'],
 'Cabal Ritual': [2, 1, 'instant', 3],
 'Candelabra of Tawnos': [1, 1, 'artifact', 0],
 'Chain of Vapor': [1, 1, 'instant', 0],
 'Chrome Mox': [0, 1, 'artifact', 1],
 'Command Tower': [0, 1, 'land', 0],
 'Cyclonic Rift': [2, 1, 'instant', 0],
 'Dark Confidant': [2, 1, 'creature', 0],
 'Dark Ritual': [1, 1, 'instant', 2],
 'Delay': [2, 1, 'instant', 'counter'],
 'Demonic Tutor': [2, 1, 'sorcery', 'tutor'],
 'Dimir Signet': [2, 1, 'artifact', 6],
 'Dispel': [1, 1, 'instant', 'counter'],
 'Dramatic Reversal': [2, 1, 'instant', 'combo'],
 'Enlightened Tutor': [1, 1, 'instant', 'ttutor'],
 'Etherium Sculptor': [2, 1, 'creature', 0],
 'Fellwar Stone': [2, 1, 'artifact', 6]

In [219]:
def makedeck(stat):
    deck = [];
    for i in stat.keys():
        for x in range(stat[i][1]):
            deck.append(i)
    return deck

makedeck(cardattr)

['Ancient Tomb',
 'Arid Mesa',
 'Bloodstained Mire',
 'Chrome Mox',
 'Command Tower',
 'Flooded Strand',
 'Hallowed Fountain',
 'Island',
 'Island',
 'Island',
 'Island',
 'Island',
 'Island',
 'Island',
 'Island',
 'Island',
 "Lion's Eye Diamond",
 'Lotus Petal',
 'Mana Crypt',
 'Marsh Flats',
 'Misty Rainforest',
 'Mox Diamond',
 'Mox Opal',
 'Pact of Negation',
 'Polluted Delta',
 'Prairie Stream',
 'Scalding Tarn',
 'Scrubland',
 'Sunken Hollow',
 'Tundra',
 'Underground Sea',
 'Urborg, Tomb of Yawgmoth',
 'Verdant Catacombs',
 'Watery Grave',
 'Windswept Heath',
 "Angel's Grace",
 'Brainstorm',
 'Candelabra of Tawnos',
 'Chain of Vapor',
 'Dark Ritual',
 'Dispel',
 'Enlightened Tutor',
 'Flusterstorm',
 'Gitaxian Probe',
 'High Tide',
 'Imperial Seal',
 'Mana Vault',
 'Mental Misstep',
 'Mystical Tutor',
 'Nihil Spellbomb',
 'Ponder',
 'Preordain',
 'Quicken',
 'Retract',
 "Sensei's Divining Top",
 'Silence',
 'Sol Ring',
 'Spell Pierce',
 'Swan Song',
 'Swords to Plowshares',
 'V

In [238]:
def iswin(hand, stat):
    htype = [stat[i][3] for i in hand]
    tut = htype.count('tutor')
    com = htype.count('combo')
    top = htype.count('ttutor')
    draw = htype.count('draw')
    coun = htype.count('counter')
    mana = sum([x for x in htype if isinstance(x, int)])
    if tut + com >= 2 and mana > 1 and coun > 0:
        return True
    elif top>0 and draw>0 and tut+com+top>=2 and mana > 1 and coun > 0:
        return True
    else:
        return False

In [243]:
def adnauseam(life, cards, stat):
    deck = makedeck(stat)
    hand = []
    mana = 0
    count = 0
    random.shuffle(deck)
    deck.remove('Ad Nauseam')
    for draw in range(cards):
        deck.pop();
    while life > max([stat[i][0] for i in deck]):
        card = deck.pop()
        hand.append(card)
        life -= stat[card][0]
        if isinstance(stat[card][3], int) and stat[card][3] < 5:
            mana += stat[card][3]
        count += 1;
    return [hand, mana, count, iswin(hand, stat)]

adnauseam(30, 15, cardattr)

[['Flusterstorm',
  'Mystical Tutor',
  'Paradoxical Outcome',
  'Dark Confidant',
  'Prairie Stream',
  'Island',
  'Dark Ritual',
  'Snapcaster Mage',
  'Underground Sea',
  'Frantic Search',
  "Jace, Vryn's Prodigy",
  'Scroll Rack',
  'Retract',
  'Urborg, Tomb of Yawgmoth',
  'Dimir Signet',
  'Isochron Scepter',
  'Misty Rainforest',
  'Mana Drain',
  'Marsh Flats',
  "Night's Whisper"],
 2,
 20,
 True]

In [244]:
def repeatan(n, life, cards, stat):
    results = []
    for i in range(n):
        results.append(adnauseam(30, 15, cardattr))
    return results

In [248]:
test = repeatan(5000, 30, 15, cardattr)
[i[3] for i in test].count(True)

2603

In [223]:
b = [1,1,2,3,4,5]
b.count(1)

2

In [93]:
d = {}
d['a'] = 5
d['a'] = 6
d

{'a': 6}

In [85]:
cards[0]

<a class="qty board" data-board="main" data-category="0-drop" data-ihash="4618396d46407f4cc9bc70681d59a5e0a6c1bebd" data-name="Ancient Tomb" data-orig="Ancient Tomb" data-qty="1" data-slug="ancient-tomb" href="#" id="board-4618396d46407f4cc9bc70681d59a5e0a6c1bebd">1x</a>

In [97]:
cardattr

{'Ad Nauseam': [5, 1],
 'Aetherflux Reservoir': [4, 1],
 'Ancient Tomb': [0, 1],
 "Angel's Grace": [1, 1],
 'Arid Mesa': [0, 1],
 'Azorius Signet': [2, 1],
 'Bloodstained Mire': [0, 1],
 'Brainstorm': [1, 1],
 'Cabal Ritual': [2, 1],
 'Candelabra of Tawnos': [1, 1],
 'Chain of Vapor': [1, 1],
 'Chrome Mox': [0, 1],
 'Command Tower': [0, 1],
 'Cyclonic Rift': [2, 1],
 'Dark Confidant': [2, 1],
 'Dark Ritual': [1, 1],
 'Delay': [2, 1],
 'Demonic Tutor': [2, 1],
 'Dimir Signet': [2, 1],
 'Dispel': [1, 1],
 'Dramatic Reversal': [2, 1],
 'Enlightened Tutor': [1, 1],
 'Etherium Sculptor': [2, 1],
 'Fellwar Stone': [2, 1],
 'Flooded Strand': [0, 1],
 'Flusterstorm': [1, 1],
 'Force of Will': [5, 1],
 'Frantic Search': [3, 1],
 'Gitaxian Probe': [1, 1],
 'Grasp of Fate': [3, 1],
 'Grim Monolith': [2, 1],
 'Hallowed Fountain': [0, 1],
 'Helm of Awakening': [2, 1],
 'High Tide': [1, 1],
 "Hurkyl's Recall": [2, 1],
 'Imperial Seal': [1, 1],
 'Impulse': [2, 1],
 'Island': [0, 9],
 'Isochron Scepte